In [4]:
import cv2
import imutils
import numpy as np
import glob

In [3]:
index = 0
cam = cv2.VideoCapture(0)
while True:
    ret, frame = cam.read()
    frame2 = imutils.resize(frame, width=1000)
    cv2.imshow("Video", frame2)
    h, w, _ = frame.shape
    h2, w2, _ = frame2.shape
    key = cv2.waitKey(1) & 0xFF
    if key == ord("c"):
#         print(w)
#         print(w2)
#         print("save")
        print(index)
        #cv2.imwrite("images/camera_calibration/checkerboard_img_" + str(index) + ".jpg", frame2)
        cv2.imwrite("images/camera_calibration/checkerboard_img_w"  + str(w) + "_"+ str(index) + ".jpg", frame)
        cv2.imwrite("images/camera_calibration/checkerboard_img_w" + str(w2) + "_" + str(index) + ".jpg", frame2)
        index = index + 1
    if key == ord("q"):
        cv2.destroyWindow("Video")
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28


In [5]:
# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,4,0)
objp = np.zeros((5*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:5].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('images/camera_calibration/checkerboard_img_w1000_*.jpg')

In [7]:
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,5),None)

    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)

        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (7,5), corners2,ret)
        cv2.imshow('img',img)
        cv2.waitKey(500)

In [6]:
cv2.destroyAllWindows()

In [ ]:
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)

In [8]:
print("Camera matrix: {}".format(mtx))
print("Distortion coefficients: {}".format(dist))
# print("Calibration matrix values: {}".format(cv2.calibrationMatrixValues(mtx, )))
print("Rotation Vectors: {}".format(rvecs))
print("Translation Vectors: {}".format(tvecs))

Camera matrix: [[ 587.79833276    0.          487.59051853]
 [   0.          588.46112685  304.02582498]
 [   0.            0.            1.        ]]
Distortion coefficients: [[-0.0265891   0.12327877  0.00112269 -0.00323499 -0.11906654]]
Rotation Vectors: [array([[-0.00100074],
       [ 0.01631426],
       [-0.0235615 ]]), array([[-0.36288984],
       [-0.0074155 ],
       [ 0.10323975]]), array([[ 0.06651402],
       [ 0.75400489],
       [ 0.09268634]]), array([[ 0.18862985],
       [-0.68945179],
       [ 0.00417327]]), array([[-0.52332262],
       [ 0.05069336],
       [ 0.14691784]]), array([[ 0.60530367],
       [-0.00950272],
       [ 0.07952514]]), array([[ 0.29372493],
       [-0.58019181],
       [ 0.06234755]]), array([[-0.37520138],
       [ 0.61494986],
       [ 0.1190888 ]]), array([[ 0.46536616],
       [ 0.67567598],
       [ 0.21749373]]), array([[-0.36551242],
       [ 0.03325084],
       [ 0.23230342]]), array([[ 0.26452565],
       [ 0.46230112],
       [ 0.016710

In [16]:
img = cv2.imread('images/camera_calibration/checkerboard_img_w1000_0.jpg')
img2 = cv2.imread('images/camera_calibration/checkerboard_img_w1000_0.jpg')
#img3 = cv2.imread('images/camera_calibration/checkerboard_img_w1000_3.jpg')
h,  w = img.shape[:2]
newcameramtx, roi=cv2.getOptimalNewCameraMatrix(mtx,dist,(w,h),1,(w,h))

In [17]:
# undistort
dst = cv2.undistort(img, mtx, dist, None, newcameramtx)
dst2 = cv2.undistort(img, mtx, dist)

# crop the image
x,y,w,h = roi
#dst = dst[y:y+h, x:x+w]
cv2.imshow("undistorted", img2)
cv2.imshow("Result", dst)
cv2.imshow("Result no new mtx", dst2)

cv2.waitKey(0)

13

In [18]:
print(frame2.shape)

(562, 1000, 3)


In [19]:
np.savez('calibration_values', mtx=mtx, dist=dist, rvecs=rvecs, tvecs=tvecs, shape=frame2.shape)